In [ ]:
!pip install -U --quiet ultralytics albumentations wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 62.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 5.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 3.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━

In [ ]:
import os, yaml, torch, random, numpy as np
from ultralytics import YOLO
from albumentations import (
    Compose, RandomSizedBBoxSafeCrop,RandomBrightnessContrast, HueSaturationValue, MotionBlur,
    GaussNoise, CoarseDropout, CLAHE, HorizontalFlip, VerticalFlip,
    Affine, OneOf, Resize
)
import shutil
from pathlib import Path

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
DATA_ROOT = "/kaggle/input/dataset/merged_dataset_2"

TRAIN_IMAGES = f"{DATA_ROOT}/train/images"
VAL_IMAGES   = f"{DATA_ROOT}/valid/images"

In [ ]:
print("CUDA:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
else:
    print("⚠️  Chạy trên CPU, sẽ rất chậm!")

CUDA: True
GPU: Tesla P100-PCIE-16GB


In [ ]:
import torch.nn as nn

class CoordAtt(nn.Module):
    def __init__(self, channels, reduction=16):
        super().__init__()
        mip = max(8, channels // reduction)

        self.pool_h = nn.AdaptiveAvgPool2d((None, 1))
        self.pool_w = nn.AdaptiveAvgPool2d((1, None))

        self.conv1 = nn.Conv2d(channels, mip, kernel_size=1, stride=1, padding=0)
        self.bn1 = nn.BatchNorm2d(mip)
        self.act = nn.SiLU()

        self.conv_h = nn.Conv2d(mip, channels, kernel_size=1, stride=1, padding=0)
        self.conv_w = nn.Conv2d(mip, channels, kernel_size=1, stride=1, padding=0)

    def forward(self, x):
        identity = x
        n, c, h, w = x.size()

        x_h = self.pool_h(x)
        x_w = self.pool_w(x).permute(0, 1, 3, 2)

        y = torch.cat([x_h, x_w], dim=2)
        y = self.act(self.bn1(self.conv1(y)))

        x_h, x_w = torch.split(y, [h, w], dim=2)
        x_w = x_w.permute(0, 1, 3, 2)

        a_h = torch.sigmoid(self.conv_h(x_h))
        a_w = torch.sigmoid(self.conv_w(x_w))

        return identity * a_h * a_w


In [ ]:
import ultralytics.nn.modules as nn_mod
import ultralytics.nn.tasks as y_tasks

nn_mod.CoordAtt = CoordAtt
y_tasks.CoordAtt = CoordAtt
globals()['CoordAtt'] = CoordAtt

In [ ]:
%%writefile yolov11s_ca_alb.yaml

nc: 5
depth_multiple: 1.0
width_multiple: 1.0

anchors:
  - [12,16, 19,36, 40,28]
  - [36,75, 76,55, 72,146]
  - [142,110, 192,243, 459,401]

backbone:
  - [-1, 1, Conv, [64, 3, 2]]
  - [-1, 1, Conv, [128, 3, 2]]
  - [-1, 3, C3, [128]]
  - [-1, 1, CoordAtt, [128]]
  - [-1, 1, Conv, [256, 3, 2]]
  - [-1, 6, C3, [256]]
  - [-1, 1, CoordAtt, [256]]
  - [-1, 1, Conv, [512, 3, 2]]
  - [-1, 6, C3, [512]]
  - [-1, 1, SPPF, [512, 5]]

head:
  - [-1, 3, C3, [512]]
  - [-1, 1, Conv, [256, 1, 1]]
  - [-1, 1, nn.Upsample, [None, 2, 'nearest']]
  - [[-1, 5], 1, Concat, [1]]
  - [-1, 3, C3, [256]]
  - [-1, 1, Conv, [128, 1, 1]]
  - [-1, 1, nn.Upsample, [None, 2, 'nearest']]
  - [[-1, 2], 1, Concat, [1]]
  - [-1, 3, C3, [128]]
  - [-1, 1, Conv, [128, 3, 2]]
  - [[-1, 14], 1, Concat, [1]]
  - [-1, 3, C3, [256]]
  - [-1, 1, Conv, [256, 3, 2]]
  - [[-1, 10], 1, Concat, [1]]
  - [-1, 3, C3, [512]]
  - [[8, 11, 14], 1, Detect, [nc]]


Writing yolov11s_ca_alb.yaml


In [ ]:
def get_train_transforms(imgsz=640):
    return Compose([
        RandomSizedBBoxSafeCrop(width=imgsz, height=imgsz, erosion_rate=0.2, p=0.3),
        Resize(imgsz, imgsz, p=1),
        HorizontalFlip(p=0.5),
        VerticalFlip(p=0.0),
        OneOf([
            RandomBrightnessContrast(0.2, 0.2, p=0.5),
            HueSaturationValue(5, 25, 5, p=0.5)
        ], p=0.7),
        OneOf([MotionBlur(5, p=0.2), GaussNoise(p=0.2)], p=0.3),
        CLAHE(clip_limit=2.0, p=0.3),
        CoarseDropout(num_holes=8, max_h_size=64, max_w_size=64, p=0.5),
        Affine(rotate=(-10,10), translate_percent=(0.0,0.1), scale=(0.5,1.2), p=0.7)
    ], bbox_params={'format':'yolo', 'label_fields':['class_labels']})

from ultralytics.data.augment import Albumentations
Albumentations.transform = get_train_transforms
print("Custom Albumentations pipeline registered ✔️")

Custom Albumentations pipeline registered ✔️


In [ ]:
model = YOLO("/kaggle/input/model_yolov11/other/default/1/best.pt")
    data="data.yaml",
    epochs=100,
    imgsz=640,
    batch=16,
    patience=10,
    optimizer='SGD',
    lr0=0.001,
    lrf=0.001,
    momentum=0.937,
    weight_decay=0.0005,
    warmup_epochs=3,
    hsv_h=0.0,
    hsv_s=0.0,
    hsv_v=0.0,
    cos_lr=True,
    degrees=0.0,
    translate=0.0,
    scale=0.3,
    shear=0.0,
    cls=0.7,
    perspective=0.0,
    flipud=0.0,
    fliplr=0.0,
    mosaic=1.0,
    mixup=0.05,
    copy_paste=0.0,
    project='runs',
    name='yolov11_ca_alb_custom',
    device=0,
    pretrained=False,
    verbose=False,
    resume=True
)

Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.7, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/kaggle/input/dataset/merged_dataset_2/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.0, hsv_s=0.0, hsv_v=0.0, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.001, mask_ratio=4, max_det=300, mixup=0.05, mode=train, model=/kaggle/input/model_yolov11/other/default/1/best.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov11_ca_alb_custom, nbs=64, nms=False, opset=None, optimize=False, optimizer=Ada

100%|██████████| 755k/755k [00:00<00:00, 3.76MB/s]



                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  3    156928  ultralytics.nn.modules.block.C3              [128, 128, 3]                 
  3                  -1  1      3352  CoordAtt                                     [128]                         
  4                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  5                  -1  6   1118208  ultralytics.nn.modules.block.C3              [256, 256, 6]                 
  6                  -1  1     12848  CoordAtt                                     [256]                         
  7                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256

100%|██████████| 5.35M/5.35M [00:00<00:00, 16.2MB/s]


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.1±0.2 ms, read: 17.4±3.5 MB/s, size: 52.2 KB)


...

  0%|          | 0/703 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/autograd/graph.py:823: UserWarning: adaptive_avg_pool2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:92.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
     36/100      6.29G       1.15     0.9003      1.187         46        640: 100%|██████████| 703/703 [06:02<00:00,  1.94it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 113/113 [00:39<00:00,  2.83it/s]

                   all       1808      20635      0.676      0.692      0.641      0.429



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/703 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/autograd/graph.py:823: UserWarning: adaptive_avg_pool2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:92.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
     37/100      6.27G      1.179     0.9291      1.211         88        640:  78%|███████▊  | 548/703 [04:42<01:20,  1.94it/s]


KeyboardInterrupt: 

In [ ]:
model = YOLO("/kaggle/working/runs/yolov11_ca_alb_custom/weights/best.pt")

import torch
from packaging import version
if version.parse(torch.__version__) >= version.parse("2.0"):
    model.model = torch.compile(model.model)

    print("✅ Model compiled for faster inference")

metrics = model.val(
    data="/kaggle/input/dataset/merged_dataset_2/data.yaml",
    split="test"
)

✅ Model compiled for faster inference
Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLOv11s_ca_alb summary (fused): 135 layers, 28,948,647 parameters, 0 gradients, 178.7 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 6.3±1.7 MB/s, size: 52.6 KB)


val: Scanning /kaggle/input/dataset/merged_dataset_2/test/labels... 877 images, 4 backgrounds, 0 corrupt: 100%|██████████| 877/877 [00:04<00:00, 179.13it/s]

WARNING ⚠️ val: Cache directory /kaggle/input/dataset/merged_dataset_2/test is not writeable, cache not saved.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 55/55 [00:21<00:00,  2.50it/s]
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        877       9406      0.631      0.751      0.633      0.404
                   bus        142        180      0.692      0.728      0.683      0.446
           truck large        336        824      0.732      0.722      0.716      0.451
          truck medium        343       1197      0.746      0.683      0.716      0.464
           truck small         19         31      0.126      0.871      0.233      0.168
                   car        861       7174      0.857      0.749      0.819       0.49
Speed: 0.4ms preprocess, 20.8ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/val


In [ ]:
!zip -r /kaggle/working/runs/yolov11_ca_alb_custom.zip /kaggle/working/runs/yolov11_ca_alb_custom

  adding: kaggle/working/runs/yolov11_ca_alb_custom/ (stored 0%)
  adding: kaggle/working/runs/yolov11_ca_alb_custom/BoxP_curve.png (deflated 8%)
  adding: kaggle/working/runs/yolov11_ca_alb_custom/results.csv (deflated 61%)
  adding: kaggle/working/runs/yolov11_ca_alb_custom/train_batch1.jpg (deflated 1%)
  adding: kaggle/working/runs/yolov11_ca_alb_custom/confusion_matrix_normalized.png (deflated 20%)
  adding: kaggle/working/runs/yolov11_ca_alb_custom/BoxPR_curve.png (deflated 8%)
  adding: kaggle/working/runs/yolov11_ca_alb_custom/confusion_matrix.png (deflated 20%)
  adding: kaggle/working/runs/yolov11_ca_alb_custom/val_batch1_pred.jpg (deflated 8%)
  adding: kaggle/working/runs/yolov11_ca_alb_custom/val_batch2_labels.jpg (deflated 8%)
  adding: kaggle/working/runs/yolov11_ca_alb_custom/train_batch2.jpg (deflated 1%)
  adding: kaggle/working/runs/yolov11_ca_alb_custom/val_batch0_labels.jpg (deflated 7%)
  adding: kaggle/working/runs/yolov11_ca_alb_custom/BoxR_curve.png (deflated 9